In [31]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pformat

from hloc import extract_features, match_features, localize_inloc, visualization

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pipeline for indoor localization

## Setup
Here we declare the paths to the dataset, image pairs, and we choose the feature extractor and the matcher. You need to download the [InLoc dataset](https://www.visuallocalization.net/datasets/) and put it in `datasets/inloc/`, or change the path.

In [32]:
dataset = Path("datasets/inloc/")  # change this if your dataset is somewhere else

pairs = Path("pairs/inloc/")
loc_pairs = pairs / "pairs-ross300.txt"  # top 40 retrieved by NetVLAD

outputs = Path("outputs/inloc/")  # where everything will be saved
results = outputs / "InLoc_hloc_superpoint+superglue_netvlad40.txt"  # the result file

In [33]:
# list the standard configurations available
print(f"Configs for feature extractors:\n{pformat(extract_features.confs)}")
print(f"Configs for feature matchers:\n{pformat(match_features.confs)}")

Configs for feature extractors:
{'d2net-ss': {'model': {'multiscale': False, 'name': 'd2net'},
              'output': 'feats-d2net-ss',
              'preprocessing': {'grayscale': False, 'resize_max': 1600}},
 'dir': {'model': {'name': 'dir'},
         'output': 'global-feats-dir',
         'preprocessing': {'resize_max': 1024}},
 'disk': {'model': {'max_keypoints': 5000, 'name': 'disk'},
          'output': 'feats-disk',
          'preprocessing': {'grayscale': False, 'resize_max': 1600}},
 'eigenplaces': {'model': {'name': 'eigenplaces'},
                 'output': 'global-feats-eigenplaces',
                 'preprocessing': {'resize_max': 1024}},
 'netvlad': {'model': {'name': 'netvlad'},
             'output': 'global-feats-netvlad',
             'preprocessing': {'resize_max': 1024}},
 'openibl': {'model': {'name': 'openibl'},
             'output': 'global-feats-openibl',
             'preprocessing': {'resize_max': 1024}},
 'r2d2': {'model': {'max_keypoints': 5000, 'name': 'r

In [34]:
# pick one of the configurations for extraction and matching
# you can also simply write your own here!
feature_conf = extract_features.confs["superpoint_inloc"]
matcher_conf = match_features.confs["superglue"]

## Extract local features for database and query images

In [30]:
feature_path = extract_features.main(feature_conf, dataset, outputs)

[2024/10/29 16:47:21 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2024/10/29 16:47:21 hloc INFO] Found 7119 images in root datasets/lamar.
[2024/10/29 16:47:23 hloc INFO] Skipping the extraction.


## Match the query images
Here we assume that the localization pairs are already computed using image retrieval (NetVLAD). To generate new pairs from your own global descriptors, have a look at `hloc/pairs_from_retrieval.py`. These pairs are also used for the localization - see below.

In [25]:
match_path = match_features.main(
    matcher_conf, loc_pairs, feature_conf["output"], outputs
)

[2024/10/29 16:42:28 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


  0%|                                                                                                                                                        | 0/123480 [00:00<?, ?it/s]


AssertionError: Missing key keypoints0 in data

## Localize!
Perform hierarchical localization using the precomputed retrieval and matches. Different from when localizing with Aachen, here we do not need a 3D SfM model here: the dataset already has 3D lidar scans. The file `InLoc_hloc_superpoint+superglue_netvlad40.txt` will contain the estimated query poses.

In [7]:
localize_inloc.main(
    dataset, loc_pairs, feature_path, match_path, results, skip_matches=20
)  # skip database images with too few matches

[2024/10/28 17:38:06 hloc INFO] Starting localization...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:39<00:00,  2.63s/it]
[2024/10/28 17:38:45 hloc INFO] Writing poses to outputs/inloc/InLoc_hloc_superpoint+superglue_netvlad40.txt...
[2024/10/28 17:38:45 hloc INFO] Writing logs to outputs/inloc/InLoc_hloc_superpoint+superglue_netvlad40.txt_logs.pkl...
[2024/10/28 17:38:45 hloc INFO] Done!


Couldn't localize query/iphone7/IMG_0994.JPG
Couldn't localize query/iphone7/IMG_1107.JPG


## Visualization
We parse the localization logs and for each query image plot matches and inliers with a few database images.

In [9]:
visualization.visualize_loc(results, dataset, n=2, top_k_db=2, seed=1)

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/inloc/InLoc_hloc_superpoint+superglue_netvlad40.txt_logs.pkl'